#### RF for datasets

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time


B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")
A = pd.read_pickle("~/bge-analysis-simv3/A.pkl")

from sklearn.ensemble import RandomForestRegressor


import os

In [2]:
rep = 6 

In [3]:
tableg=A.T
species_ones = [] 
for species in range(16):
    species_ones.append(tableg[tableg[species]==1.0][species].index.to_list())

In [4]:
samplist = [327]

In [5]:
max_depthl = [2,5,10,15,20]
ntrees = [100,200,300,400,500,1000]
max_featuresl = ["auto","sqrt","log2"]

In [6]:
rfper =  [] 
listt = list(np.arange(0,2**15,1))
for species in range(16):
    for replica in range(rep,rep+1):
        steadystate = pd.read_pickle("~/compressed_sensingv1/old-data/fullsearch/15_2.3_"+str(replica)+"/steady-state.pkl")
        n = 2**16
        sst = steadystate.T
        sstv2 = sst.copy()
        sstv2["total"] = sstv2.T.sum()
        for i in range(16):
            sstv2[i] = sstv2[i]/sstv2["total"]
        sstv3 = sstv2.fillna(0)
        abunvec = pd.DataFrame(sstv3[species].values[species_ones[species]])
        abunvec.index = np.arange(0,2**15,1)
        for m in samplist:
            ri = pd.read_pickle("~/compressed_sensingv1/abun_simv3/training-data/sp"+str(species)+"_rep"+str(replica)+"-in-sample-"+str(m)+".pkl")    
            onescomms =  tableg.T[species_ones[species]].T.index 
            actcomms = pd.DataFrame(onescomms).T[ri].T[0].values
            xtrain = tableg.T[actcomms].T
            ytrain = abunvec.T[ri].T[0].values
            
            tedata = list(set(list(np.arange(0,2**15,1))).difference(ri))
            tecomms = pd.DataFrame(onescomms).T[tedata].T[0].values
            
            xte = tableg.T[tecomms].T
            yte = abunvec.T[tedata].T[0].values
            starttt = time.time()
            for md in max_depthl:
                for n_est in ntrees:
                    for mfeatures in max_featuresl:

                        regr = RandomForestRegressor(max_depth=md,n_estimators=n_est,max_features=mfeatures)
                        regr.fit(xtrain,ytrain)
                        ypred = regr.predict(xte)

                        rfper.append([replica,species,md,n_est,mfeatures,ypred,time.time() - starttt])
            

/home/shreyaa5/miniconda3/envs/csense/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/shreyaa5/miniconda3/envs/csense/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/shreyaa5/miniconda3/envs/csense/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or re

In [7]:
rdf = pd.DataFrame(rfper)
rdf.to_pickle("rep"+str(replica)+"RFREL.pkl")

In [8]:
rdf

,0,1,2,3,4,5,6
0,6,0,2,100,auto,"[0.03352379782740793, 0.03352379782740793, 0.0...",0.143990
1,6,0,2,100,sqrt,"[0.025944772517387485, 0.02586030986265451, 0....",0.275879
2,6,0,2,100,log2,"[0.026297679622010853, 0.02613809640695922, 0....",0.412341
3,6,0,2,200,auto,"[0.03254548609285552, 0.03254548609285552, 0.0...",0.690954
4,6,0,2,200,sqrt,"[0.027146020114696964, 0.02701417000300868, 0....",0.945753
...,...,...,...,...,...,...,...
1435,6,15,20,500,sqrt,"[0.29264704194951713, 0.2862782651715109, 0.28...",54.409345
1436,6,15,20,500,log2,"[0.2948419040967775, 0.30210410105728597, 0.29...",55.280386
1437,6,15,20,1000,auto,"[0.44463521113011306, 0.4384960409438307, 0.44...",57.050348
1438,6,15,20,1000,sqrt,"[0.30730748228815574, 0.3094540567498643, 0.29...",58.780579
